In [64]:
import pandas as pd
from enum import Enum
import re
import random
import spacy
from spacy.util import minibatch
from spacy.training import Example
from spacy.scorer import Scorer

In [65]:
prqt = pd.read_parquet('./parquet/MultL/test_multilingual.parquet', engine='fastparquet')

In [66]:
prqt.columns

Index(['tokens', 'ner_tags', 'lang'], dtype='object')

In [68]:


def corrigir_espacos(texto: str) -> str:
    """
    Corrige espaçamento incorreto em frases de português/espanhol/inglês.
    Regras básicas:
    - Nenhum espaço antes de pontuação (. , ; : ? ! …)
    - Um espaço depois de pontuação, exceto antes de fechar aspas/parênteses
    - Espaço antes de abrir parênteses/colchetes/chaves
    - Nenhum espaço depois de abrir ou antes de fechar parênteses/colchetes/chaves
    """

    # Remove espaços antes de sinais de pontuação
    texto = re.sub(r'\s+([.,;:!?…])', r'\1', texto)

    # Garante um espaço depois de sinais de pontuação (se não for fim de frase ou antes de aspas/fechamentos)
    texto = re.sub(r'([.,;:!?…])([^\s"\'\)\]\}])', r'\1 \2', texto)

    # Espaço antes de abrir parênteses/colchetes/chaves, se não houver
    texto = re.sub(r'(\w)([\(\[\{])', r'\1 \2', texto)

    # Remove espaço logo após abrir parênteses/colchetes/chaves
    texto = re.sub(r'([\(\[\{])\s+', r'\1', texto)

    # Remove espaço logo antes de fechar parênteses/colchetes/chaves
    texto = re.sub(r'\s+([\)\]\}])', r'\1', texto)

    # Aspas: remove espaço depois de abrir aspas
    texto = re.sub(r'([“"\'«])\s+', r'\1', texto)

    # Aspas: remove espaço antes de fechar aspas
    texto = re.sub(r'\s+([”"\'»])', r'\1', texto)

    # Espaço depois de fechar aspas, se seguido de palavra
    texto = re.sub(r'([”"\'»])([A-Za-zÁ-ú])', r'\1 \2', texto)

    return texto


In [69]:
prqt['phase'] = prqt['tokens'].apply(lambda x: corrigir_espacos(" ".join(x)))

In [70]:
prqt['dict'] = prqt.apply(
    lambda row: dict(zip(row['tokens'], row['ner_tags'])),
    axis=1
)

prqt['tup'] = prqt.apply(
    lambda row: (row['tokens'], row['ner_tags']),
    axis=1
)


In [73]:
# Classificação conforme os intervalos
intervalos = [
    (range(1, 3), "PER"),
    (range(3, 5), "ORG"),
    (range(5, 7), "LOC"),
    (range(7, 9), "MIST"),
]

def classificar(valor: int) -> str:
    for intervalo, significado in intervalos:
        if valor in intervalo:
            return significado
    return None

# Função principal que opera sobre a tupla (tokens, tags)
def processar_tupla(tupla):
    tokens, tags = tupla
    resultado = []
    grupo = []
    label_atual = None

    for palavra, valor in zip(tokens, tags):
        if valor == 0:
            # Fecha o grupo se tiver algo
            if grupo:
                frase = corrigir_espacos(" ".join(grupo))
                resultado.append((frase, label_atual))
                grupo = []
                label_atual = None
            continue
        
        if valor % 2 == 1:
            # Novo grupo
            if grupo:
                frase = corrigir_espacos(" ".join(grupo))
                resultado.append((frase, label_atual))
            grupo = [palavra]
            label_atual = classificar(valor)
        
        elif valor % 2 == 0 and grupo:
            grupo.append(palavra)

    # Fecha o último grupo
    if grupo:
        frase = corrigir_espacos(" ".join(grupo))
        resultado.append((frase, label_atual))
    
    return resultado


In [74]:
prqt["substring_labeled"] = prqt["tup"].apply(processar_tupla)

In [75]:
def get_position(text: str, sub: str) -> tuple | None:
    begin = text.find(sub)
    if begin == -1:
        return None
    end = begin + len(sub)
    return begin, end


In [76]:
positions = []

# Itera sobre as linhas do DataFrame
for _, row in prqt.iterrows():
    l = []
    for sub in row["substring_labeled"]:
        texto_entidade = sub[0]   # a frase extraída
        label = sub[1]            # o label: "PER", "ORG", etc.
        begin, end = get_position(text=row["phase"], sub=texto_entidade)
        l.append((begin, end, label))
    positions.append(l)

# Adiciona a nova coluna ao DataFrame
prqt["position"] = positions

In [78]:
def tem_none(posicoes):
    return any(pos[0] is None or pos[1] is None for pos in posicoes)

# Cria uma coluna booleana temporária
prqt["has_none"] = prqt["position"].apply(tem_none)

# Conta as linhas
linhas_com_none = prqt["has_none"].sum()
linhas_sem_none = len(prqt) - linhas_com_none

print(f"❌ Linhas com pelo menos um (None, None): {linhas_com_none}")
print(f"✅ Linhas com todos os offsets válidos: {linhas_sem_none}")


❌ Linhas com pelo menos um (None, None): 0
✅ Linhas com todos os offsets válidos: 31375


In [79]:
list_finished_dataset = []

for idx, row in prqt.iterrows():
    list_finished_dataset.append((row["phase"], {"entities": row["position"]}))

In [80]:
cut = int(len(list_finished_dataset) * 0.1)

DEV_DATA = list_finished_dataset[:cut]
TRAIN_DATA = list_finished_dataset[cut:]

In [ ]:
# -----------------------------
# 1) Dados mock (corrigidos)
# -----------------------------
TRAIN_DATA = [
    ("João mora em São Paulo.", {"entities": [(0, 4, "PER"), (13, 22, "LOC")]}),
    ("Maria trabalha na Google em Belo Horizonte.", {"entities": [(0, 5, "PER"), (18, 24, "ORG"), (28, 42, "LOC")]}),
    ("A Apple comprou a Embraer.", {"entities": [(2, 7, "ORG"), (18, 25, "ORG")]}),
    ("A Ana visitou o Rio de Janeiro.", {"entities": [(2, 5, "PER"), (16, 30, "LOC")]}),
    ("Pedro nasceu em Lisboa.", {"entities": [(0, 5, "PER"), (16, 22, "LOC")]}),
    ("A Microsoft abriu escritório em Recife.", {"entities": [(2, 11, "ORG"), (32, 38, "LOC")]}),
]

DEV_DATA = [
    ("Carla foi para Porto Alegre ontem.", {"entities": [(0, 5, "PER"), (15, 27, "LOC")]}),
    ("Google contratou João em 2024.", {"entities": [(0, 6, "ORG"), (17, 21, "PER")]}),
]

In [ ]:
# --------------------------------------
# 2) Cria um pipeline NER do zero (pt)
# --------------------------------------
nlp = spacy.blank("pt")           # modelo em branco (sem vocabulário treinado)
ner = nlp.add_pipe("ner")

# adiciona os rótulos vistos nos dados
for _, ann in TRAIN_DATA + DEV_DATA:
    for start, end, label in ann.get("entities", []):
        ner.add_label(label)

# helper para converter (text, ann) -> Example
def make_examples(nlp, data):
    examples = []
    for text, ann in data:
        doc = nlp.make_doc(text)
        spans = []
        for start, end, label in ann["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")

            if span is None:
                raise ValueError(f"Span inválido em: {text!r} -> {(start, end, label)}")
            
            spans.append(span)
            
        doc_ents = {"entities": [(s.start_char, s.end_char, s.label_) for s in spans]}
        examples.append(Example.from_dict(doc, doc_ents))
    return examples

train_examples = make_examples(nlp, TRAIN_DATA)
dev_examples   = make_examples(nlp, DEV_DATA)

ValueError: Span inválido em: 'The lake also has an endemic species flock of amphipods consisting of 11" Hyalella"(an additional Titicaca" Hyalella" species is nonendemic).' -> (74, 82, 'LOC')

In [ ]:
# --------------------------------------
# 3) Treinamento
# --------------------------------------
# desabilite outros pipes (aqui só temos o 'ner' mesmo)
other_pipes = [p for p in nlp.pipe_names if p != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.initialize(get_examples=lambda: train_examples)
    n_iter = 30
    for itn in range(1, n_iter + 1):
        random.shuffle(train_examples)
        losses = {}
        # minibatches progressivamente maiores ajudam em dados pequenos
        for batch in minibatch(train_examples, size=4):
            nlp.update(batch, sgd=optimizer, drop=0.2, losses=losses)
        if itn % 5 == 0 or itn == 1 or itn == n_iter:
            print(f"Iter {itn:02d} - loss: {losses.get('ner', 0):.4f}")

In [ ]:
# 📌 Célula 4 – Avaliação (corrigida)
def evaluate(nlp, examples):
    # gera previsões de forma vetorizada
    pred_docs = list(nlp.pipe([ex.text for ex in examples]))
    pred_examples = [Example(pred, ex.reference) for pred, ex in zip(pred_docs, examples)]

    scorer = Scorer()
    scores = scorer.score(pred_examples)  # <-- passa a LISTA de Example
    return {
        "precision": scores["ents_p"],
        "recall":    scores["ents_r"],
        "f1":        scores["ents_f"],
    }

metrics = evaluate(nlp, dev_examples)
print("DEV metrics:", metrics)


In [ ]:

# --------------------------------------
# 5) Teste rápido
# --------------------------------------
tests = [
    "Marcos works on Apple in São Paulo.",
    "the Embraer is locaded in Brasil.",
    "Ana fly to new york.",
]
for t in tests:
    doc = nlp(t)
    print("\nTexto:", t)
    for ent in doc.ents:
        print(f" - {ent.text:<20} {ent.label_}")


In [ ]:
# 📌 Célula 7 – Salvar o modelo treinado
output_dir = "modelo_ner_en"
nlp.to_disk(output_dir)
print(f"Modelo salvo em: {output_dir}")


In [ ]:
# 📌 Célula 8 – Carregar o modelo treinado
import spacy
nlp_carregado = spacy.load("modelo_ner_en")

# Teste rápido
doc = nlp_carregado("Tom went to Paris and met with Alice.")
for ent in doc.ents:
    print(ent.text, ent.label_)
